In [ ]:
#importing necessary libraries
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import seaborn
import warnings
warnings.filterwarnings("ignore",category= DeprecationWarning)
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
#Reading the data
data = pd.read_csv('/kaggle/input/reddit-india-flair-detection/datafinal.csv')

dataf = data.copy()
data

# Data Cleaning

#### Dropping unwanted columns.

In [ ]:
data.columns

In [ ]:
data = data.drop(['score','url','comms_num','author','timestamp'],axis=1)

In [ ]:
data.head()

In [ ]:
data['title'][0]

In [ ]:
data['body'][0]

In [ ]:
data['combined_features'][0]

In [ ]:
data['comments'][0]

##### Note that combined features column is actually combination text of three columns as title,comments, url and combined_features.

I am dropping that column too for ease of analysis.

In [ ]:
data = data.drop(['combined_features'],axis=1)
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

There are 18 unique flairs.

In [ ]:
data['flair'].unique()

Note that some of the flairs are date values. Let's explore it a bit to see what it is...

In [ ]:
data.groupby('flair')['title'].describe()

In [ ]:
data.groupby('flair')['id'].describe()

In [ ]:
data.groupby('flair')['body'].describe()

In [ ]:
data.groupby('flair')['comments'].describe()

Seems like there is no significant details present related to those 'date-time' flairs. So let's drop those entries.

In [ ]:
#Dropping the rows corresponding to date-time flairs
f = data['flair'].dropna()
regx = re.compile(r"[\d]{1,2}-[\d]{1,2}-[\d]{4} [\d]{1,2}:[\d]{1,2}")
for __ in f:
    #print(flair)
    x = regx.search(__)
    if x is not None:
        #print(x.group())
        d = data[data.flair == x.group()]
        #print(d)
        data = data.drop(d.index)

In [ ]:
data['flair'].unique()

There are some nan values in flair. Exploring it....

In [ ]:
data[data['flair'] == np.nan].describe()

##### Note: There are no significant data related to nan flair value. Hence we can drop it.

In [ ]:
data = data.dropna(subset=['flair'])

In [ ]:
data.info()

##### Combine [title,body and comments] for text-processing

In [ ]:
data['text'] = data['title'].astype(str) + data['body'].astype(str) + data['comments'].astype(str)

In [ ]:
data_final = data[['flair','id','text']]
data_final.head()

In [ ]:
data_final.describe()

In [ ]:
data_final.groupby('flair')['text'].describe()

##### Removing Special characters

In [ ]:
data_final['text'] = data_final['text'].str.replace("[^a-zA-Z0-9 \n.]"," ")

In [ ]:
"""Now we have clean data!!!"""
data_final.head(10) 

# Text Preprocessing

### Removing Punctuation, Stopwords and Tokenization

In [ ]:
"""
    1. Removing all punctuation
    2. Removing stop-words
    3. Returns a clean text
"""
def clean_txt(mess):
    
    nonpunc = [char for char in mess if char not in string.punctuation] #list of strings which are non-punc
    
    nonpunc = "".join(nonpunc) #join back to form the whole string
    
    return [word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]    

In [ ]:
data_final['text'] = data_final['text'].apply(clean_txt)

### Lemmatization

In [ ]:
from nltk import WordNetLemmatizer

le = WordNetLemmatizer()

data_final['text'] = data_final['text'].apply(lambda x : [le.lemmatize(word) for word in x])
data_final['text'] = data_final['text'].apply(lambda x : " ".join(x))
data_final['text']

In [ ]:
qwe = data_final['text'].copy()
qwe

### Vectorization

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer = clean_txt).fit(data_final['text'])

In [ ]:
text_bow = bow_transformer.transform(data_final['text'])

In [ ]:
print(f'Shape of sparse matrix is {text_bow.shape}')
print(f'Length of dictionary is {len(bow_transformer.vocabulary_)}')
print(f'Number of non=zero occusrances is {text_bow.nnz}')
sparsity = (text_bow.nnz/(text_bow.shape[0]*text_bow.shape[1]))*100
print('Sparsity :', sparsity)

### TfIdf

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer().fit(text_bow)
text_tfidf = tfidf_transformer.transform(text_bow)

In [ ]:
print(f'Shape of tfidf of text is {text_tfidf.shape}')

# Traning a Model (Naive Bayes)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB().fit(text_tfidf,data_final['flair'])

In [ ]:
predictions = model.predict(text_tfidf)

# Model Evaluation

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(data_final['flair'],predictions))

# Prediction 

## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_txt)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB()),
])

# Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
text_train, text_test, flair_train, flair_test = train_test_split(data_final['text'],data_final['flair'])

In [ ]:
pipeline.fit(text_train,flair_train)

In [ ]:
predictions = pipeline.predict(text_test)

In [ ]:
print(classification_report(flair_test,predictions))

##### Conclusion: Devised a model which can predict flair if given a text.

### Input 

In [ ]:
print(text_test)

### Prediction

In [ ]:
ids = [data_final.iloc[int(i)]['id'] for i in text_test.index]
Predicted_df = pd.DataFrame({'ID':ids,'Text':text_test,'PredictedFlair':predictions}).reset_index(drop=True)

In [ ]:
Predicted_df